# Implementacija Neuronske Mreže

* Implementiraj neuronsku mrežu. 
* Mreža mora imati mehanizam za dodavanje slojeva. 
* Sloj mora imati aktivacijsku funkciju.
* Mrežu je neophodno trenirati koristeći gradijentni spust, koristeći odgovarajuću loss (cost) funkciju.
* Neuronska mreža mora imati metod `predict`.

In [93]:
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import random
from sympy import diff

In [94]:
class FCLayer:
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.input = None
        self.output = None
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation
        self.activation_prime = activation_prime

        self.weight = np.random.rand(output_size, input_size)
        self.bias = np.random.rand(output_size,1)
        

    def forward_propagation(self, x):   
        self.ulaz = x
        self.y = np.dot(self.weight, x) + self.bias
        z = self.activation(self.y)
        return z
        

    def backward_propagation(self, loss, lr):
        delta = loss *self.activation_prime(self.y)

        self.weight = self.weight - lr*delta*self.ulaz.T
        self.bias = self.bias - lr*delta
        loss_output = np.dot(self.weight.T, loss)
        return loss_output

        

In [95]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

In [96]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))/2

def mse_prime(y_true, y_pred):
    return (y_pred-y_true)/y_true.size

In [97]:
class Network:

    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
    
    def add(self,layer):
        self.layers.append(layer)

    def set(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def fit(self, x_train, y_train, epochs, lr):
        samples = len(x_train)
        br=1
        while br <= epochs:
            j = random.choice(range(samples))
            output = x_train[j]
            for layer in self.layers:
                output = layer.forward_propagation(output)


            error =mse_prime(y_train[j], output)

            for layer in reversed(self.layers):
                error = layer.backward_propagation(error,lr) 
                
           

            if br % 100 == 0:
                print(mse(y_train[j],output))

            br = br + 1

    

    def predict(self, x_test):
        samples = len(x_test)
        result = [] 

        for i in range(samples):
            output = x_test[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    
    

In [98]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [99]:
# ucitavanje MNIST skupa
(x_train, y_train), (x_test, y_test) = mnist.load_data() # train = 60k, test = 10k primjera


# pripremiti trening skup podataka
# reshape + normalization
x_train = x_train.reshape(x_train.shape[0], 28*28, 1)
x_train = x_train.astype('float32')
x_train /= 255


# one-hot encoding
# kodirati rezultate (brojeve iz opsega [0,9]) u vektor velicine 10
# npr. broj 3 se predstavlja kao vektro [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)
y_train = y_train.reshape(y_train.shape[0], 10,1)

# pripremiti test skup - isto kao i za trening
x_test = x_test.reshape(x_test.shape[0], 28*28, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
y_test = y_test.reshape(y_test.shape[0],10,1)

print(x_test.shape)
print(y_test.shape)

(10000, 784, 1)
(10000, 10, 1)


Instanciraj neuronsku mrežu, sa sigmoidnom funkciojom kao aktivacionom funkcijom.

Za loss funkciju koristi MSE, istreniraj mrežu sa learning_rate = 0.1 i 600000 epoha, kreiraj predikcije i ispiši classification report...

In [100]:
# NN iz prvog primjera
net = Network()

net.add(FCLayer(28*28, 15, sigmoid, sigmoid_prime))

net.add(FCLayer(15, 10, sigmoid, sigmoid_prime))

In [106]:
net.set(mse, mse_prime)
net.fit(x_train, y_train, 100000, 0.001)

0.3993266635664949
0.4001241951217323
0.39944824885177826
0.3993300058125282
0.39939244698208054
0.3993215199105081
0.399390452429233
0.39934083465488346
0.3993886425133619
0.3993208912780673
0.399440324384499
0.4893785866784438
0.3993360535137387
0.3993144374543265
0.3993350367246191
0.3993338084938981
0.3993162734708748
0.3993114535116479
0.39930823567740803
0.3993804630687897
0.3993125628465249
0.3993085367714549
0.3993261389156064
0.3993754807483288
0.39932284260820283
0.3993009413185618
0.3993000285506003
0.39930534889150404
0.39931957699568643
0.3993029656425636
0.48937804656911243
0.39929733452591154
0.40012043425175836
0.40011990357185356
0.3993640591389241
0.4001193514964531
0.4893771309437097
0.3992873998809752
0.3993080697637619
0.3993578650765834
0.3994079482338747
0.4893807673983397
0.399302576144276
0.39930123308105114
0.39940381050584783
0.3993519003044223
0.399277454386692
0.3993464033817963
0.4893669776384142
0.39934583713893923
0.3992749370156761
0.3992768071247875
0.

In [102]:
predictions = net.predict(x_test)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
accuracy_score(y_test,to_categorical(np.argmax(np.reshape(np.array(predictions), (-1,10)), axis=1)))


In [ ]:
print(classification_report(y_test, to_categorical(np.argmax(np.reshape(np.array(predictions), (-1,10)), axis=1))))

Kreiraj mrežu komplikovanije strukture (više slojeva), istreniraj sa istim parametrima kao u prethodnom slučaju i uporedi rezultate.

In [ ]:
# malo kompleksnija mreza
net = Network()

net.add(FCLayer(28*28, 256, sigmoid, sigmoid_prime))

net.add(FCLayer(256, 64, sigmoid, sigmoid_prime))

net.add(FCLayer(64, 32, sigmoid, sigmoid_prime))

net.add(FCLayer(32, 10, sigmoid, sigmoid_prime))

In [ ]:
net.set(mse, mse_prime)
net.fit(x_train, y_train, 1000, 0.01)

In [ ]:
predictions = net.predict(x_test)

In [ ]:
print(classification_report(y_test, to_categorical(np.argmax(np.reshape(np.array(predictions), (-1,10)), axis=1))))